In [1]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch
import random
import math 
import copy
import random
import argparse
import torch.optim as optim
import torch.nn as nn
import modeldefine
import numpy as np
from scipy.optimize import minimize
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
alpha = 0.01

def UCB(A, phi):
    #### ucb term
    phi = phi.view(-1,1)
    try:
        tmp, LU = torch.linalg.solve(phi,A)
    except:
        A = A.detach().numpy()
        phi2 = phi.detach().numpy()
        tmp = torch.Tensor(np.linalg.solve(A, phi2))

    return torch.sqrt(torch.matmul(torch.transpose(phi,1,0), tmp))

def calculate_v(contextinfo_list, A, theta):
    vj_list = []
    feature_list = []
    for i in contextinfo_list:
        feature = model(i.to(device)).cpu()
        first_item =  torch.mm( feature.view(1,-1) , theta)
        second_item = alpha * UCB(A, feature)
        vj_list.append((first_item + second_item).item())
        feature_list.append(feature.detach().numpy())
    return np.array(vj_list), feature_list

def update_A(A, info_subset):
    for i in info_subset:
        i = torch.tensor(i, dtype=torch.float32,device=device)
        feature = model(i.to(device)).view(1,-1).cpu()
        A = A + torch.mm(feature.t(), feature)
    return A

# 这里调小utility 
def prob(vj_list):
    sum = np.sum(np.exp(vj_list)) + 1
    return [np.exp(vj_list[i]) / sum for i in range(len(vj_list))]  

def revenue(vj_list, reward_list):
    sum = np.sum(np.exp(vj_list)) + 1
    return np.sum(np.multiply(np.exp(vj_list), reward_list) / sum)

def assort(contextinfo_list, reward_list, vj_list, feature_list):
    length = len(vj_list)
    # sort the contextinfo_list and vj with descending order of reward_list
    sorted_list = sorted(zip(contextinfo_list, vj_list, reward_list, feature_list), key=lambda x: x[2], reverse=True)
    
    contextinfo_list = [x[0] for x in sorted_list]
    vj_list = [x[1] for x in sorted_list]
    reward_list = [x[2] for x in sorted_list]
    feature_list = [x[3] for x in sorted_list]

    # calculate the optimal assortment
    optimal_assort = []
    optimal_reward = 0
    index = 1
    for i in range(length):
        if revenue(vj_list[:index], reward_list[:index]) >= optimal_reward:
            optimal_reward = revenue(vj_list[:index], reward_list[:index])
            index += 1
        else:
            break
    return contextinfo_list[:index], feature_list[:index], vj_list[:index], reward_list[:index]

# this is for the linear purchase model when v = x dot theta
def get_linear_purchase(feature_list):
    true_Vlist = [(TRUE_THETA @ feature_list[i].reshape(-1,1)).item() for  i in range(len(feature_list))]
    prob_list = prob(true_Vlist)

    # sample item according to prob_list
    if random.uniform(0,1) < 1 - np.sum(prob_list):
        return np.array([0 for i in range(len(feature_list))])
    else:
        returnlist = [0 for i in range(len(feature_list))]
        indexchoose = random.choices([i for i in range(len(prob_list))], weights = prob_list)[0]
        returnlist[indexchoose] = 1
        return np.array(returnlist)

In [3]:
lambd = 1
def likelihood(theta, feature_list ,y_list):
    # feature's dimension is len * dimension , theta is 1*dimension
    v_list = np.matmul(feature_list, theta.T).reshape(-1)
    ln_prob = np.log(prob(v_list))
    summation = ln_prob * y_list
    return -1 * np.sum(summation)

def likelihood_derivative(theta, feature_list, y_list):
    v_list = np.matmul(feature_list, theta.T).reshape(-1)
    prob_list = prob(v_list)
    summation = np.matmul(np.array(feature_list).T, (y_list - prob_list))
    return -1 * summation

def likelihood_array(theta, feature_list_list, y_list_list):
    summation =  0.5 * lambd * np.dot(theta, theta)
    for i in range(len(feature_list_list)):
        summation += likelihood(theta, feature_list_list[i], y_list_list[i])
    return summation

def likelihood_derivative_array(theta, feature_list_list, y_list_list):
    summation = 0.5 * lambd * theta
    for i in range(len(feature_list_list)):
        summation += likelihood_derivative(theta, feature_list_list[i], y_list_list[i])
    return summation

In [4]:
class CustomLikelihoodLoss(nn.Module): 
    def __init__(self, theta_list):
        super(CustomLikelihoodLoss, self).__init__()
        self.theta_list = theta_list

    def forward(self, output_list, y_list):
        loss = 0
        index = 0
        for output in output_list:  
            y = torch.tensor(y_list[index]).to(device) .view(-1,1)
            theta = torch.tensor(self.theta_list[index], dtype= torch.float32).to(device) 
            v = torch.mm(output, theta.view(-1,1)) 
            prob = torch.exp(v) / (torch.sum(torch.exp(v)) + 1)  
            loss += torch.sum(torch.log(prob) * y)  
            index += 1  
        return -loss 

class CustomLikelihoodLoss2(nn.Module):
    def __init__(self, theta_list):
        super(CustomLikelihoodLoss2, self).__init__()
        self.theta_list = theta_list
     
    def forward(self, output_list, y_list):
        loss = 0
        index = 0
        for output in output_list:
            y = torch.tensor(y_list[index]).to(device).view(-1,1)
            theta = torch.tensor(self.theta_list[index], dtype= torch.float32).to(device)
            v = torch.mm(output, theta.view(-1,1))
            prob = torch.exp(v) / (torch.sum(torch.exp(v)) + 1)
            # ce loss between prob and y
            loss += torch.sum(-y * torch.log(prob) - (1-y) * torch.log(1-prob))
            index += 1
        return loss 


In [5]:
# this block is only used for linear model revenue calculation
def calculate_linear_v(contextinfo_list, A, theta):
    vj_list = []
    feature_list = []
    for i in contextinfo_list:
        feature = i
        first_item =  torch.mm( feature.view(1,-1) , theta)

        vj_list.append(first_item.item())
        feature_list.append(feature.detach().numpy())
    return np.array(vj_list), feature_list

# 真实情况乱下的theta，feature
def get_true_linear_ass(context, profit):
    theta_tensor = torch.tensor(TRUE_THETA.reshape(-1,1), dtype=torch.float32)
    v_array,initial_feature = calculate_linear_v(torch.tensor(context,dtype=torch.float32), LAMBDA, theta_tensor)
    assortment, ass_features, v_list, profit_list = assort(context, profit.tolist()[0], v_array.tolist() , initial_feature)
    true_probablility = np.array(prob(v_list))
    revenue = np.dot(true_probablility, np.array(profit_list))
    return revenue


def get_assortment_revenue(assortment, profit):
    theta_tensor = torch.tensor(TRUE_THETA.reshape(-1,1), dtype=torch.float32)
    v_array,initial_feature = calculate_linear_v(torch.tensor(assortment,dtype=torch.float32), LAMBDA, theta_tensor)
    true_probablility = np.array(prob(v_array))
    revenue = np.dot(true_probablility, np.array(profit))
    return revenue

In [6]:
import modeldefine
import torch.optim as optim
model = modeldefine.Model(5,10,10,2).to(device)
# 10 20 20 20 20 20  5
optimizer = optim.Adam(model.parameters(), lr=0.01,weight_decay=0.01)

tensor([ 5, 10, 10, 10], dtype=torch.int32)


In [7]:
print(model)

Model(
  (layers): ModuleList(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [8]:
# data reader
CONTEXT_ARRAY = np.load('linear_data/features.npy') 
REWARD_ARRAY = np.load('linear_data/rewards.npy')
TRUE_THETA = np.load('linear_data/theta.npy')

In [9]:
data_length = len(CONTEXT_ARRAY)

# define the hyperparameters
input_size = 20
hidden_size = 20
output_size = 10
num_layers = 10

beta = 0.1

H = 100

# initialize the parameters

theta = np.random.randn(output_size) / np.sqrt(output_size)

LAMBDA = lambd * torch.eye(output_size, dtype=torch.float32)

ass_list = []
feature_list = []
purchase_list = []
theta_list = []

revenue_list1 = []
revenue_list2 = []
true_profit_list = []

for t in range(0,1000):
    context = CONTEXT_ARRAY[t]
    profit = REWARD_ARRAY[t]

    theta_tensor = torch.tensor(theta.reshape(-1,1), dtype=torch.float32)
    v_array,initial_feature = calculate_v(torch.tensor(context,dtype=torch.float32), LAMBDA, theta_tensor)
    assortment, ass_features, vv_list , reward = assort(context, profit.tolist()[0], v_array.tolist() , initial_feature)
    purchase_vector = get_linear_purchase(assortment)
    
    # calculate the ideal 
    expected_revenue1 = get_true_linear_ass(context, profit)
    revenue_list1.append(expected_revenue1)
    true_profit_list.append(np.dot(np.array(purchase_vector), reward))
    revenue_list2.append(get_assortment_revenue(assortment, reward))

    # add to list
    ass_list.append(np.array(assortment))
    feature_list.append(np.array(ass_features))
    purchase_list.append(purchase_vector)

    # update the parameters
    LAMBDA = update_A(LAMBDA, assortment)
    
    # update theta using MLE
    initial_guess = theta
    result = minimize(likelihood_array, initial_guess, args=(ass_list, purchase_list), method='SLSQP', 
                  constraints={'type':'eq', 'fun': likelihood_derivative_array, 'args':(ass_list, purchase_list)})
    theta = result.x
    theta_list.append(theta)

    # update the neural networks
    if t % H == 99:
        a_list = ass_list[-1*H:]
        y_list = purchase_list[-1*H:]

        loss_function = CustomLikelihoodLoss2(theta_list)
        epochs = 1
     
        for epoch in range(epochs):
            output_list = [model(torch.tensor(a,dtype = torch.float32).to(device)) for a in a_list]
            loss = loss_function(output_list, y_list)
            if (epoch == epochs-1): print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}")
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        theta_list = []

C:\Users\alienware\AppData\Local\Temp\ipykernel_10216\2392795184.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:264.)
  v_array,initial_feature = calculate_linear_v(torch.tensor(assortment,dtype=torch.float32), LAMBDA, theta_tensor)


Epoch [1/1], Loss: 284.0516662597656
Epoch [1/1], Loss: 242.5410614013672
Epoch [1/1], Loss: 225.7886962890625
Epoch [1/1], Loss: 207.48240661621094
Epoch [1/1], Loss: 199.094482421875
Epoch [1/1], Loss: 187.4322509765625
Epoch [1/1], Loss: 195.0421142578125
Epoch [1/1], Loss: 183.36216735839844
Epoch [1/1], Loss: 184.12957763671875
Epoch [1/1], Loss: 181.85052490234375


In [11]:
np.save('linear_data/record/true_revenue_list1.npy', np.array(revenue_list1))
np.save('linear_data/record/simulate_revenue_list2.npy', np.array(revenue_list2))
np.save('linear_data/record/purchase_revenue_list.npy', np.array(true_profit_list))

In [11]:
print(prob((ass_list[100] @ TRUE_THETA.reshape(10,1)).reshape(1,-1)))
print(prob(calculate_v(torch.tensor(ass_list[100],dtype=torch.float32), LAMBDA, theta_tensor)[0]))

[array([0.24755697, 0.17920725, 0.10858287, 0.22995762, 0.22108807])]
[0.23925270797584142, 0.20596848007044882, 0.08502806381887187, 0.23441803513736983, 0.2289798856667121]


In [12]:
print(prob((ass_list[99] @ TRUE_THETA.reshape(10,1)).reshape(1,-1)))
print(prob(calculate_v(torch.tensor(ass_list[99],dtype=torch.float32), LAMBDA, theta_tensor)[0]))


[array([0.10034455, 0.21076841, 0.14508133, 0.17142846, 0.35271862])]
[0.09658509735433717, 0.18352291563725892, 0.12234222012826149, 0.16023457541176545, 0.42805959006122735]


In [15]:
purchase_list[99]

array([0, 1, 0, 0, 0])

In [ ]:
# regret calculate
# ult - 1  让不购买概率最大
# matolotlib linear  10000 epoch 
# 真实情况下的expected revenue， theta， Truetheta
# expected 和 assortment 的revenue 顾客的purchase 
# 


